In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
jd = pd.read_csv('job_data_dict_all_weighted.csv')


In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 43336 entries, 0 to 43335
Data columns (total 5 columns):
Unnamed: 0    43336 non-null int64
jobs          43336 non-null object
jobtitle      43336 non-null object
dictionary    43336 non-null object
id            43336 non-null int64
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [4]:
jd.head()

,Unnamed: 0,jobs,jobtitle,dictionary,id
0,1,supervisors of office and administrative staff,client success account coordinator,client success account coordinator client succ...,1
1,2,logisticians,account coordinator supply chain customer service,account coordinator supply chain customer serv...,2
2,3,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...,3
3,4,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...,4
4,5,sales reps wholesale and manufacturing,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...,5


In [5]:

cat_columns = ['jobs','jobtitle','dictionary']
feature_columns = ['jobtitle','dictionary']   
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,jobs,jobtitle,dictionary
0,supervisors of office and administrative staff,client success account coordinator,client success account coordinator client succ...
1,logisticians,account coordinator supply chain customer service,account coordinator supply chain customer serv...
2,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
3,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
4,sales reps wholesale and manufacturing,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...


In [6]:
jd_2.jobs.value_counts().min()

30

In [7]:
df_numerical = jd_2.copy()
encoder_dict = {}

for col in string_variables:
    le = preprocessing.LabelEncoder()
    le.fit(jd_2[col])
    encoder_dict[col] = le
    df_numerical[col] = le.transform(jd_2[col])

df_numerical.head()

,jobs,jobtitle,dictionary
0,157,client success account coordinator,client success account coordinator client succ...
1,89,account coordinator supply chain customer service,account coordinator supply chain customer serv...
2,119,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
3,119,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
4,144,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...


In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_2['dictionary'], jd_2['jobs'], random_state=1)

In [9]:
#feat = list(df_numerical.dictionary.values.flatten())
#type(feat)
#feat = unicode(feat, 'utf-8')

In [10]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
y_test


array(['medical secretaries', 'travel agents',
       'education administrators postsecondary', ..., 'production workers',
       'secretaries not legal or medical',
       'demonstrators and product promoters'], dtype=object)

In [11]:
X_test.shape

(10834L,)

In [12]:
X_train.shape

(32502L,)

In [13]:
type(X_train)

numpy.ndarray

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [15]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

<32502x14392 sparse matrix of type '<type 'numpy.int64'>'
	with 343223 stored elements in Compressed Sparse Row format>

In [16]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

<10834x14392 sparse matrix of type '<type 'numpy.int64'>'
	with 112458 stored elements in Compressed Sparse Row format>

In [17]:
train_features = vect.get_feature_names()
len(train_features)

14392

In [18]:
#train_features[:50]

In [19]:
#train_features[-50:]

In [20]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_arr, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
7644,15392,manager
11563,12964,senior
3705,10005,director
4321,8440,engineer
11278,8136,sales
1003,6013,assistant
1783,5634,business
7717,5377,marketing
11616,4930,service
8963,4865,operations


In [22]:
# use SVM
from sklearn import svm
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm, y_train)
#Predict 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [23]:
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm)
preds

array(['financial managers branch or department', 'travel agents',
       'education administrators postsecondary', ...,
       'assemblers and fabricators',
       'supervisors of office and administrative staff',
       'demonstrators and product promoters'], dtype=object)

In [24]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.740539043751
[[183   0   0 ...,   0   0   0]
 [  0 130   0 ...,   0   0   0]
 [  0   0   2 ...,   0   0   0]
 ..., 
 [  0   0   0 ..., 159   0   0]
 [  0   0   0 ...,   0  12   0]
 [  0   0   0 ...,   0   0   1]]


In [25]:
# predict (poorly calibrated) probabilities and calculate AUC
probs = svc.predict_proba(test_dtm)[:, 1]
probs
#print(metrics.roc_auc_score(y_test, probs))

array([ 0.06044742,  0.00127611,  0.00209352, ...,  0.00433471,
        0.06385416,  0.00386791])

In [26]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [27]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [28]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

accountants       0.78      0.79      0.78       233
administrative assistants       0.83      0.76      0.80       170
administrative services managers       0.29      0.25      0.27         8
advertising and promotions managers       0.45      0.62      0.53         8
advertising sales agents       0.82      0.55      0.66        42
art directors       0.70      0.89      0.78        18
assemblers and fabricators       0.42      0.47      0.44        17
audio and video equipment technicians       0.58      0.88      0.70         8
   auditors       0.77      0.79      0.78        52
automotive master mechanics       0.73      0.80      0.76        10
automotive specialty technicians       0.91      0.77      0.83        13
 bartenders       0.79      0.85      0.81        13
bill and account collectors       0.92      0.92      0.92        49
billing cost and rate clerks       0.75      0.89      0.81        37
biological technici

In [29]:

target_predicted = svc.predict(test_dtm)
target_prob = svc.predict_proba(test_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(y_test, target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,administrative services managers,advertising and promotions managers,advertising sales agents,art directors,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,...,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,wholesale and retail buyers except farm products,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,183,0,0,0,0,0,0,0,0,0,...,0,11,0,0,0,0,0,0,0,233
administrative assistants,0,130,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,170
administrative services managers,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
advertising and promotions managers,0,0,0,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
advertising sales agents,0,0,0,0,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,42
art directors,0,0,0,0,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
assemblers and fabricators,0,0,0,0,0,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,17
audio and video equipment technicians,0,0,0,0,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,8
auditors,2,0,0,0,0,0,0,0,41,0,...,0,1,0,0,0,0,0,0,0,52


In [30]:
from sklearn.cross_validation import cross_val_score
cross_val_score(svc, train_dtm, y_train, cv=5, scoring='accuracy').mean()

0.58852424402317072

In [31]:
# try using all of the data

In [32]:
train_dtm = vect.fit_transform(jd_2['dictionary'])


In [33]:
# train a SVC model using train_dtm
svc = svm.SVC(kernel='linear', probability = True)


svc.fit(train_dtm, jd_2['jobs'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
preds = svc.predict(train_dtm)
preds.shape

(43336L,)

In [35]:

print(classification_report(jd_2['jobs'], preds) )

             precision    recall  f1-score   support

accountants       0.73      0.82      0.77       881
administrative assistants       0.78      0.84      0.81       657
administrative services managers       0.76      0.83      0.79        53
advertising and promotions managers       0.52      0.87      0.65        46
advertising sales agents       0.67      0.56      0.61       149
art directors       0.85      0.34      0.49        67
assemblers and fabricators       0.74      0.90      0.81        79
audio and video equipment technicians       0.86      0.75      0.80        32
   auditors       0.85      0.76      0.80       220
automotive master mechanics       0.71      0.63      0.67        43
automotive specialty technicians       0.72      0.91      0.80        56
 bartenders       0.96      0.80      0.87        59
bill and account collectors       0.84      0.75      0.79       246
billing cost and rate clerks       0.88      0.55      0.68       171
biological technici

In [36]:

target_predicted = svc.predict(train_dtm)
target_prob = svc.predict_proba(train_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_2['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,administrative services managers,advertising and promotions managers,advertising sales agents,art directors,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,...,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,wholesale and retail buyers except farm products,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,719,1,0,0,0,0,0,0,3,0,...,0,6,0,0,0,0,0,0,0,881
administrative assistants,7,551,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,657
administrative services managers,0,0,44,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,53
advertising and promotions managers,0,0,0,40,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,46
advertising sales agents,0,0,0,0,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,149
art directors,0,0,0,6,0,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,67
assemblers and fabricators,0,0,0,0,0,0,71,0,0,0,...,0,0,0,0,0,0,0,0,0,79
audio and video equipment technicians,0,0,0,0,0,0,0,24,0,0,...,0,0,0,0,0,0,0,0,0,32
auditors,9,1,0,0,0,0,0,0,167,0,...,0,0,0,0,0,0,0,0,0,220


In [37]:
from sklearn.cross_validation import cross_val_score
cross_val_score(svc, train_dtm, jd_2['jobs'], cv=5, scoring='accuracy').mean()

0.5192397685752479

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.grid_search import GridSearchCV

In [ ]:
# I dont need to do pca on this 
pipeline = Pipeline([('svm', svm.SVC(C=1.0, kernel='linear'))])
param_grid = {
    
    'svm__kernel': ['rbf', 'linear', 'poly','sigmoid']
}
clf = GridSearchCV(pipeline, param_grid)
clf = clf.fit(train_dtm, jd_2['jobs'])
clf.best_params_